# SENAI / INEP - Dados sobre evasão
## Explicação do arquivo CSV
---

Este arquivo CSV está relacionado a demanda da Gerência de Educação: LUMNI - Assistente virtual.

Um dos objetivos deste trabalho foi a previsão de evasão de alunos do SENAI-PR. Dados dados históricos de um grupo de alunos, como dados sobre notas em avaliações, frequência, condições socioeconômicas e outros, a tarefa seria determinar o potencial de evasão de um aluno com determinadas características.

Como a gerência não disponibilizou nenhum dado (até o fim da primeira sprint), os residentes buscaram dados disponibilizados pelo INEP do Censo da Educação Superior, disponível na página de Microdados (http://inep.gov.br/microdados). A partir dos dados do censo de 2018, foi realizada uma filtragem para obter os dados de alunos das unidades do SENAI-PR. Após isso, foi realizada uma limpeza dos dados, eliminando informações que foram consideradas irrelevantes para os objetivos propostos.

---

In [4]:
import pandas as pd

In [5]:
senai = pd.read_csv('senai_inep.csv')

In [6]:
senai.shape

(3309, 22)

In [7]:
senai.head()

,CO_IES,NO_IES,CO_CURSO,NO_CURSO,TP_TURNO,TP_GRAU_ACADEMICO,TP_MODALIDADE_ENSINO,ID_ALUNO,TP_COR_RACA,TP_SEXO,...,IN_DEFICIENCIA,DT_INGRESSO_CURSO,IN_RESERVA_VAGAS,IN_FINANCIAMENTO_ESTUDANTIL,IN_APOIO_SOCIAL,IN_ATIVIDADE_EXTRACURRICULAR,TP_ESCOLA_CONCLUSAO_ENS_MEDIO,IN_MATRICULA,IN_CONCLUINTE,TP_SITUACAO
0,1400,FACULDADE DA INDÚSTRIA SÃO JOSÉ DOS PINHAIS,20516,ADMINISTRAÇÃO,3.0,1.0,1,C2E870F652B2299ED62D4AA8038A9422,1,1,...,0,01JUL2009:00:00:00,0,0.0,0,0,1,0,0,0
1,1400,FACULDADE DA INDÚSTRIA SÃO JOSÉ DOS PINHAIS,20516,ADMINISTRAÇÃO,3.0,1.0,1,81A4D85417E7123C8DF075A0A2629E8E,0,1,...,0,01JUL2009:00:00:00,0,0.0,0,0,1,1,1,1
2,1400,FACULDADE DA INDÚSTRIA SÃO JOSÉ DOS PINHAIS,20516,ADMINISTRAÇÃO,3.0,1.0,1,77F900FA4B592D61029032DCA66D9F55,2,2,...,0,01JAN2014:00:00:00,0,0.0,0,0,1,1,1,1
3,1400,FACULDADE DA INDÚSTRIA SÃO JOSÉ DOS PINHAIS,20516,ADMINISTRAÇÃO,3.0,1.0,1,4D21A9481680ED505C359CE9A0F8F212,0,2,...,0,01JAN2006:00:00:00,0,0.0,0,0,9,0,0,1
4,1400,FACULDADE DA INDÚSTRIA SÃO JOSÉ DOS PINHAIS,20516,ADMINISTRAÇÃO,3.0,1.0,1,63341AACBCB526FE3A929666DE581731,0,1,...,0,01JAN2010:00:00:00,0,0.0,0,0,1,0,0,0


In [8]:
senai.columns

Index(['CO_IES', 'NO_IES', 'CO_CURSO', 'NO_CURSO', 'TP_TURNO',
       'TP_GRAU_ACADEMICO', 'TP_MODALIDADE_ENSINO', 'ID_ALUNO', 'TP_COR_RACA',
       'TP_SEXO', 'NU_IDADE', 'TP_NACIONALIDADE', 'IN_DEFICIENCIA',
       'DT_INGRESSO_CURSO', 'IN_RESERVA_VAGAS', 'IN_FINANCIAMENTO_ESTUDANTIL',
       'IN_APOIO_SOCIAL', 'IN_ATIVIDADE_EXTRACURRICULAR',
       'TP_ESCOLA_CONCLUSAO_ENS_MEDIO', 'IN_MATRICULA', 'IN_CONCLUINTE',
       'TP_SITUACAO'],
      dtype='object')

**A coluna 'TP_SITUACAO' indica a situação atual da matrícula do aluno (vinculado ou desvinculado) e pode ser utilizada como coluna-alvo**. A situação "vinculado" pode indicar diversas situações, como o aluno que está cursando de fato, ou alunos que estão com a matrícula temporariamente trancada, por exemplo.

As classes desse atributo são:
* 1: aluno vinculado;
* 2: aluno desvinculado.

In [9]:
senai.TP_SITUACAO.value_counts()

1    2882
0     427
Name: TP_SITUACAO, dtype: int64

Entre as demais colunas, estão colunas descritivas da instituição, curso e aluno, e uma análise pode ser realizada para decidir quais atributos poderiam ser utilizados como características para o modelo. Entre as informações disponíveis estão a modalidade do curso, nacionalidade do aluno, informações sobre gênero, raça, modalidade de ingresso, entre outras informações.

O arquivo "Dicionаrio de Variаveis" pode ser acessado para obter as descrições dos atributos presentes no CSV.

# PARTE DANIEL

In [10]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import StratifiedKFold

In [11]:
X_1 = senai.loc[senai["TP_SITUACAO"] == 1]
X_0 = X = senai.loc[senai["TP_SITUACAO"] == 0]
X_1 = X_1[0:427]
X = pd.concat([X_0,X_1])

In [12]:
A = X["DT_INGRESSO_CURSO"]
X["DT_INGRESSO_CURSO"] = A.iloc[:len(A)].str[3] + A.iloc[:len(A)].str[4] + A.iloc[:len(A)].str[5]

le = preprocessing.LabelEncoder()
X["NO_CURSO"] = le.fit_transform(X["NO_CURSO"])
X["NO_IES"] = le.fit_transform(X["NO_IES"])
X["DT_INGRESSO_CURSO"] = le.fit_transform(X["DT_INGRESSO_CURSO"])

y = X["TP_SITUACAO"]
X = X.drop(["NO_IES", "IN_CONCLUINTE", "TP_SITUACAO", "ID_ALUNO"], axis = 1)

In [13]:
X.head()

,CO_IES,CO_CURSO,NO_CURSO,TP_TURNO,TP_GRAU_ACADEMICO,TP_MODALIDADE_ENSINO,TP_COR_RACA,TP_SEXO,NU_IDADE,TP_NACIONALIDADE,IN_DEFICIENCIA,DT_INGRESSO_CURSO,IN_RESERVA_VAGAS,IN_FINANCIAMENTO_ESTUDANTIL,IN_APOIO_SOCIAL,IN_ATIVIDADE_EXTRACURRICULAR,TP_ESCOLA_CONCLUSAO_ENS_MEDIO,IN_MATRICULA
0,1400,20516,0,3.0,1.0,1,1,1,30,1,0,1,0,0.0,0,0,1,0
4,1400,20516,0,3.0,1.0,1,0,1,31,1,0,0,0,0.0,0,0,1,0
10,1400,20516,0,3.0,1.0,1,0,2,29,1,0,1,0,0.0,0,0,2,0
11,1400,20516,0,3.0,1.0,1,1,1,37,1,0,1,0,0.0,0,0,9,0
14,1400,20516,0,3.0,1.0,1,1,1,29,1,0,0,0,0.0,0,0,1,0


In [14]:
#SELECIONANDO AS MELHORES FEATURES
X2 = SelectKBest(chi2, k=5).fit_transform(X,y)

In [15]:
#SPLIT DADOS
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size = 0.2)

In [16]:
#SEM CROSS VALIDATION
model = SGDClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Acuracia: %.2f%%" % (model.score(X_test, y_test)*100.0)) 

Acuracia: 88.30%


In [17]:
#COM CROSS VALIDATION GRADIENTE DESCENDENTE
kFold = KFold(n_splits = 10, random_state = 100)
model = SGDClassifier()
resultKFold = cross_val_score(model, X_train, y_train, cv = kFold)

print("scores: ", resultKFold) 

print("Acuracia: %.2f%%" % (resultKFold.mean() * 100.0)) 

scores:  [0.86956522 0.86956522 0.89855072 0.91176471 0.82352941 0.91176471
 0.80882353 0.88235294 0.47058824 0.86764706]
Acuracia: 83.14%


In [18]:
#COM CROSS VALIDATION KNN
kFold = KFold(n_splits = 10, random_state = 100)
model = KNeighborsClassifier()
resultKFold = cross_val_score(model, X_train, y_train, cv = kFold)

print("scores: ", resultKFold) 

print("Acuracia: %.2f%%" % (resultKFold.mean() * 100.0)) 

scores:  [0.89855072 0.88405797 0.92753623 0.95588235 0.88235294 0.89705882
 0.83823529 0.94117647 0.92647059 0.92647059]
Acuracia: 90.78%


In [19]:
#COM CROSS VALIDATION DECISIONTREE
kFold = KFold(n_splits = 10, random_state = 100)
model = DecisionTreeClassifier()
resultKFold = cross_val_score(model, X_train, y_train, cv = kFold)

print("scores: ", resultKFold) 

print("Acuracia: %.2f%%" % (resultKFold.mean() * 100.0)) 

scores:  [0.89855072 0.88405797 0.95652174 0.95588235 0.88235294 0.95588235
 0.83823529 0.94117647 0.92647059 0.92647059]
Acuracia: 91.66%


In [20]:
#COM CROSS VALIDATION LINEAR REGRESSION
kFold = KFold(n_splits = 10, random_state = 100)
model = LinearRegression()
resultKFold = cross_val_score(model, X_train, y_train, cv = kFold)

print("scores: ", resultKFold) 

print("Acuracia: %.2f%%" % (resultKFold.mean() * 100.0)) 

scores:  [0.75494043 0.6689127  0.78087122 0.81798765 0.68694982 0.74451575
 0.62351699 0.73342161 0.78680183 0.78476422]
Acuracia: 73.83%


In [21]:
# ESTRATIFICACAO CROSS VALIDATION GRADIENTE DESCENDENTE
kFold = StratifiedKFold(n_splits = 10, random_state = 100)
model = SGDClassifier()
resultKFold = cross_val_score(model, X_train, y_train, cv = kFold)

print("scores: ", resultKFold) 

print("Acuracia: %.2f%%" % (resultKFold.mean() * 100.0)) 

scores:  [0.49275362 0.86956522 0.94202899 0.88405797 0.85507246 0.83823529
 0.48529412 0.88235294 0.86567164 0.86567164]
Acuracia: 79.81%


In [22]:
# ESTRATIFICACAO CROSS VALIDATION KNN
kFold = StratifiedKFold(n_splits = 10, random_state = 100)
model = KNeighborsClassifier()
resultKFold = cross_val_score(model, X_train, y_train, cv = kFold)

print("scores: ", resultKFold) 

print("Acuracia: %.2f%%" % (resultKFold.mean() * 100.0)) 

scores:  [0.86956522 0.86956522 0.97101449 0.94202899 0.92753623 0.92647059
 0.83823529 0.94117647 0.92537313 0.92537313]
Acuracia: 91.36%


In [23]:
# ESTRATIFICACAO CROSS VALIDATION DECISIONTREE
kFold = StratifiedKFold(n_splits = 10, random_state = 100)
model = DecisionTreeClassifier()
resultKFold = cross_val_score(model, X_train, y_train, cv = kFold)

print("scores: ", resultKFold) 

print("Acuracia: %.2f%%" % (resultKFold.mean() * 100.0)) 

scores:  [0.89855072 0.86956522 0.97101449 0.94202899 0.92753623 0.92647059
 0.83823529 0.94117647 0.92537313 0.92537313]
Acuracia: 91.65%


In [24]:
# ESTRATIFICACAO CROSS VALIDATION LINEARREGRESSION
kFold = StratifiedKFold(n_splits = 10, random_state = 100)
model = LinearRegression()
resultKFold = cross_val_score(model, X_train, y_train, cv = kFold)

print("scores: ", resultKFold) 

print("Acuracia: %.2f%%" % (resultKFold.mean() * 100.0)) 

scores:  [0.75291162 0.65347123 0.79467702 0.81628105 0.77517519 0.69135957
 0.62553732 0.73754128 0.79539202 0.7768675 ]
Acuracia: 74.19%


# PIPELINE

In [25]:
#modelKNN = Pipeline([('scl', StandardScaler()), ('clf', KNeighborsClassifier())])
modelDT = Pipeline([('scl', StandardScaler()), ('clf', DecisionTreeClassifier())])

rangeParam = [1, 2, 3, 4, 5]

paramGrid = [{'clf__criterion': ['gini', 'entropy'],
    'clf__min_samples_leaf': rangeParam,
    'clf__max_depth': rangeParam,
    'clf__min_samples_split': rangeParam[1:]}]

# Construct grid search
gs = GridSearchCV(estimator = modelDT, param_grid = paramGrid, scoring = 'accuracy')

# Fit using grid search
gs.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs.best_score_)

# Best params
print('\nBest params:\n', gs.best_params_)

/home/donghi/anaconda3/envs/ambDanielFiep/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best accuracy: 0.917

Best params:
 {'clf__criterion': 'gini', 'clf__max_depth': 3, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 2}


## AutoML

In [ ]:
import autosklearn.classification

automl = autosklearn.classification.AutoSklearnClassifier()
automl.fit(X_train, y_train)
y_hat = automl.predict(X_test)
print("Accuracy score", sklearn.metrics.accuracy_score(y_test, y_hat))